#### Извлечение статических эмбеддингов (TFIDF+SVD, Word2Vec+SVD)

In [3]:
import scipy
import sparsesvd

In [1]:
import numpy as np
import pandas as pd

In [2]:
from utils import *

In [ ]:
from gensim.models import Word2Vec
import gensim.downloader as gensim_downloader

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [5]:
def sparsesvd_reduce_dim(mat, n):
    return sparsesvd.sparsesvd(mat, n)[2].T
    
def scipy_svds_reduce_dim(mat, n):
    return scipy.sparse.linalg.svds(mat, k=n)[2].T

* обе функции выполняют одну и ту же работу (scipy_svds работает быстрее - использует все ядра)
* результатом SVD разложения будут 3 матрицы (используем ту, которая соответствует информации о словах)

In [7]:
def get_emb_dict(feats, mat, vec_size):
    v = scipy_svds_reduce_dim(mat.tocsc(), vec_size)
    return {feats[i]: v[i] for i in range(len(v))}

* *готовим словарь эмбеддингов (слово: эмбеддинг) из TFIDF матрицы*

In [ ]:
def get_gensim_emb_dict(m):
    k, v = m.index_to_key, m.vectors
    v = (v - v.mean(axis=0)) / v.std(axis=0)
    return {k[i]: v[i] for i in range(len(v))}

* *готовим словарь эмбеддингов (слово: эмбеддинг) из модели Word2vec*

In [8]:
%%time

params = {
    'max_df': 0.9, 
    'min_df': 0.0001, 
    'dtype': np.float32, 
    'ngram_range': (1,1), 
    'token_pattern': None, 
    'tokenizer': str.split
}

tfidf = TfidfTransformer()
cv = CountVectorizer(**params)
corpus = load_lines('data/corpus.txt')

cv_mat = cv.fit_transform(corpus)
tfidf_mat = tfidf.fit_transform(cv_mat)

cv_feats = cv.get_feature_names_out()
cv_embs = get_emb_dict(cv_feats, cv_mat, 300)
tfidf_embs = get_emb_dict(cv_feats, tfidf_mat, 300)

save_sparse(cv_mat, 'data/cv_mat.npz')
dump_pickle(cv_embs, 'data/cv_embs.bin')
save_sparse(tfidf_mat, 'data/tfidf_mat.npz')
dump_pickle(tfidf_embs, 'data/tfidf_embs.bin')
dump_lines(cv_feats, 'data/cv_feats.txt')

CPU times: user 9min 33s, sys: 5min 29s, total: 15min 2s
Wall time: 7min 12s


* *загружаем ранее подготовленный корпус текстов*
* *обучаем Count и TFIDF векторайзеры, получаем матрицы*
* *извлекаем эмбеддинги из матриц Count и TFIDF (используя SVD разложение)*
* *сохраняем матрицы, словари эмбеддингов, и сам словарь (список токенов)*

In [9]:
cv_mat.shape, cv_mat.min(), cv_mat.max()

((432158, 33187), 0.0, 83.0)

In [10]:
tfidf_mat.shape, tfidf_mat.min(), tfidf_mat.max()

((432158, 33187), 0.0, 0.9307679)

* *строим Counts и TFIDF только для униграмм*
* *подобрав min_df и max_df получаем словарь из 33187 токенов*
* *можно добавить и биграммы, но тогда придется писать более сложный алгоритм получения эмбеддинга документа*

In [11]:
len(tfidf_embs), len(next(iter(tfidf_embs.values())))

(33187, 300)

* *получаем 33187 эмбеддинга (размер словаря) по 300 фичей каждый (~41Мб на диске)*

In [8]:
%%time

params = {
    'epochs': 5, 
    'window': 5, 
    'workers': 3, 
    'negative': 5, 
    'min_count': 5, 
    'vector_size': 300, 
    'max_final_vocab': 50000,
    'corpus_file': 'data/corpus.txt'
}

m = Word2Vec(**params).wv
w2v_embs = get_gensim_emb_dict(m)
dump_pickle(w2v_embs, 'data/w2v_embs.bin')
dump_lines(m.index_to_key, 'data/w2v_feats.txt')

CPU times: user 22min 9s, sys: 4.7 s, total: 22min 13s
Wall time: 7min 59s


* *обучаем Word2Vec напрямую из файла корпуса*
* *указываем размерность эмбеддингов (300, как и для TFIDF)*
* *ограничиваем размер словаря: 50к токенов (будет подобран min_count)*

In [9]:
len(w2v_embs), len(next(iter(w2v_embs.values())))

(49448, 300)

* *получаем 49448 эмбеддинга (<50к) по 300 фичей каждый (~61Мб на диске)*

In [15]:
name = 'word2vec-ruscorpora-300'
m = gensim_downloader.load(name)
w2vp_embs = get_gensim_emb_dict(m)
dump_pickle(w2vp_embs, 'data/w2vp_embs.bin')
dump_lines(m.index_to_key, 'data/w2vp_feats.txt')

[==================================================] 100.0% 198.8/198.8MB downloaded


* *загружаем и сохраняем предобученные эмбеддинги word2vec-ruscorpora-300*

In [16]:
len(w2vp_embs), len(next(iter(w2vp_embs.values())))

(184973, 300)

* *184973 эмбеддингов (размер словаря) по 300 фичей каждый*